## Linear Regression Analysis of the relationship between BMI score, eye-sight and student score
**On CEPS data**

In [1]:
import pandas as pd
import math
from pandas.io.stata import StataReader, StataWriter

### Read Data: data1 for 2013-2014 CEPS data, data2 for 2014-2015 CEPS data

In [2]:
file = r'data/CEPS基线调查学生数据.dta'

In [3]:
stata_data = StataReader(file, convert_categoricals=False)
data = stata_data.read()
varlist = stata_data.varlist
value_labels = stata_data.value_labels()
fmtlist = stata_data.fmtlist
variable_labels = stata_data.variable_labels()

In [4]:
file2 = r'data/cepsw2studentEN.dta'

In [5]:
stata_data2 = StataReader(file2, convert_categoricals=False)
data2 = stata_data2.read()
varlist2 = stata_data2.varlist
value_labels2 = stata_data2.value_labels()
fmtlist2 = stata_data2.fmtlist
variable_labels2 = stata_data2.variable_labels()

In [6]:
print(variable_labels2)

{'ids': 'student id', 'clsids': 'class id', 'w2clsids': 'wave 2 classid', 'schids': 'school id', 'ctyids': 'county id', 'w2frame': 'which sampling frame the PSU belongs to', 'w2subsample': 'which subsampling frame the PSU belongs to', 'w2fall': 'survey time', 'w2sweight': 'wave 2 student weight', 'w1w2sweight': 'wave 1 & 2 student weight', 'w2status': 'student follow-up status', 'w2mtype': 'student missing type', 'w2mreason': 'student missing detailed reason', 'w2clsra': 'wave 2 class reassignment status', 'w2coglmh': 'wave 2 cognitive ability test-assigned category based on w1 test score', 'w2cogtype': 'wave 2 cognitive ability test-Questionnaire type the student used', 'w2cogscore': 'wave 2 cognitive ability test-raw score', 'w2cog3pl': 'wave 2 cognitive ability test-standardized score based on 3PL model', 'w2chn': '2014 fall mid-term exam raw score-Chinese', 'w2upchn': '2014 fall mid-term exam full marks-Chinese', 'w2mat': '2014 fall mid-term exam raw score-Maths', 'w2upmat': '2014 

### Filter useful data out

#### 1. Data Cleaning
The key variables we select for the study is the weight, height for 2013-2014 data and test scores
for 2014-2015 data

In [7]:
data13t14 = pd.DataFrame(data,columns=['ids','clsids','schids','ctyids','frame','subsample','fall','sweight',
                                     'stcog','cog3pl','tr_chn','tr_mat','tr_eng','a13','a14','a15','a15a','a15b'])
data13t14BMI = pd.DataFrame(data,columns=['ids','clsids','schids','ctyids','frame','subsample','fall','sweight',
                                     'a13','a14','a15'])
data14t15 = pd.DataFrame(data2,columns=['ids','clsids','schids','ctyids','w2frame','w2subsample','w2fall','w2sweight',
                                        'w2cogscore','w2cog3pl','w2chn','w2mat','w2eng','w2c01','w2c02','w2c09','w2c09a','w2c09b'])
data14t15score = pd.DataFrame(data2,columns=['ids','clsids','schids','ctyids','w2frame','w2subsample','w2fall','w2sweight',
                                        'w2cogscore','w2cog3pl','w2chn','w2mat','w2eng'])

In [8]:
data_inter = pd.merge(data13t14BMI,data14t15score,on=['ids','clsids','schids','ctyids'],how='inner')

delete nan value

In [9]:
data_nonan = data_inter.dropna(axis=0,how='any')

make poor eyesight 1 or 0

In [10]:
data_nonan['a15'].replace(2.0,1.0,inplace=True) # poor eyesight to 1
data_nonan['a15'].replace(3.0,0.0,inplace=True) # not myopia to 0

C:\Users\IT\anaconda3\envs\cs324\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


delete wrong score

In [11]:
data_ = data_nonan.loc[data_nonan['w2chn']<=100]
data_ = data_.loc[data_['w2eng']<=100]
data_ = data_.loc[data_['w2mat']<=100]
data_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5787 entries, 0 to 10278
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ids          5787 non-null   int16  
 1   clsids       5787 non-null   int16  
 2   schids       5787 non-null   int16  
 3   ctyids       5787 non-null   int8   
 4   frame        5787 non-null   int8   
 5   subsample    5787 non-null   int8   
 6   fall         5787 non-null   int8   
 7   sweight      5787 non-null   float32
 8   a13          5787 non-null   float64
 9   a14          5787 non-null   float64
 10  a15          5787 non-null   float64
 11  w2frame      5787 non-null   int8   
 12  w2subsample  5787 non-null   int8   
 13  w2fall       5787 non-null   int8   
 14  w2sweight    5787 non-null   float32
 15  w2cogscore   5787 non-null   int8   
 16  w2cog3pl     5787 non-null   float64
 17  w2chn        5787 non-null   float32
 18  w2mat        5787 non-null   float32
 19  w2eng

#### Prepare data for experiments with control variable

In [6]:
# X:
# 13t14's 身高，体重，是否近视，认知测试标准分，户口类型，是否独生子女，经济条件，教育期望，母亲教育程度，父亲教育程度
# 14t15's 希望读书到什么程度，父母婚姻状况
# Y：
# 14t15’s 期中语文，数学，英语成绩（百分制）
data13t14_ = pd.DataFrame(data,columns=['ids','clsids','schids','ctyids','stsex','a13','a14','a15','cog3pl','sthktype','stonly','steco_5c',
                                         'b31','stmedu','stfedu'])
data14t15_ = pd.DataFrame(data2,columns=['ids','clsids','schids','ctyids','w2a0801','w2b18','w2chn','w2mat','w2eng'])
data_xy = pd.merge(data13t14_,data14t15_,on=['ids','clsids','schids','ctyids'],how='inner')
data_xy.head()

,ids,clsids,schids,ctyids,stsex,a13,a14,a15,cog3pl,sthktype,stonly,steco_5c,b31,stmedu,stfedu,w2a0801,w2b18,w2chn,w2mat,w2eng
0,1,1,1,1,0,154.0,76.0,1.0,0.128507,0,1,4.0,8.0,3.0,3.0,1.0,9.0,82.0,91.0,82.0
1,2,1,1,1,1,169.0,90.0,3.0,1.235879,0,1,3.0,7.0,8.0,5.0,1.0,9.0,95.0,63.0,83.0
2,3,1,1,1,1,176.0,120.0,1.0,0.083937,1,2,3.0,7.0,3.0,3.0,1.0,7.0,89.0,68.0,74.0
3,4,1,1,1,0,160.0,90.0,1.0,-0.522185,0,1,3.0,3.0,6.0,7.0,1.0,7.0,85.0,80.0,63.0
4,5,1,1,1,0,167.0,150.0,1.0,0.036172,1,1,4.0,7.0,7.0,8.0,1.0,8.0,91.0,73.0,84.0


In [7]:
# Delete student samples with nan data
data_xy.dropna(axis=0,how='any',inplace=True)

In [8]:
# Delete samples with improper score or other variables
data_xy = data_xy.loc[data_xy['w2chn']<=100]
data_xy = data_xy.loc[data_xy['w2eng']<=100]
data_xy = data_xy.loc[data_xy['w2mat']<=100]
data_xy['a15'].replace(2.0,1.0,inplace=True) # poor eyesight to 1
data_xy['a15'].replace(3.0,0.0,inplace=True) # not myopia to 0
data_xy['a13'] = data_xy['a13'].apply(lambda x:x/100)
data_xy['a14'] = data_xy['a14'].apply(lambda x:x/2)
data_xy['BMI'] = data_xy['a14']/(data_xy['a13']*data_xy['a13'])
data_xy.head()

,ids,clsids,schids,ctyids,stsex,a13,a14,a15,cog3pl,sthktype,...,steco_5c,b31,stmedu,stfedu,w2a0801,w2b18,w2chn,w2mat,w2eng,BMI
0,1,1,1,1,0,1.54,38.0,1.0,0.128507,0,...,4.0,8.0,3.0,3.0,1.0,9.0,82.0,91.0,82.0,16.022938
1,2,1,1,1,1,1.69,45.0,0.0,1.235879,0,...,3.0,7.0,8.0,5.0,1.0,9.0,95.0,63.0,83.0,15.755751
2,3,1,1,1,1,1.76,60.0,1.0,0.083937,1,...,3.0,7.0,3.0,3.0,1.0,7.0,89.0,68.0,74.0,19.369835
3,4,1,1,1,0,1.60,45.0,1.0,-0.522185,0,...,3.0,3.0,6.0,7.0,1.0,7.0,85.0,80.0,63.0,17.578125
4,5,1,1,1,0,1.67,75.0,1.0,0.036172,1,...,4.0,7.0,7.0,8.0,1.0,8.0,91.0,73.0,84.0,26.892323


#### Descriptive Analysis

##### For only x and y

In [26]:
print(data_.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5127 entries, 0 to 10278
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ids      5127 non-null   int16  
 1   clsids   5127 non-null   int16  
 2   schids   5127 non-null   int16  
 3   ctyids   5127 non-null   int8   
 4   a13      5127 non-null   float64
 5   a14      5127 non-null   float64
 6   a15      5127 non-null   float64
 7   cog3pl   5127 non-null   float32
 8   a06      5127 non-null   float64
 9   b01      5127 non-null   float64
 10  b09      5127 non-null   float64
 11  b31      5127 non-null   float64
 12  stmedu   5127 non-null   float64
 13  stfedu   5127 non-null   float64
 14  w2a0801  5127 non-null   float64
 15  w2b18    5127 non-null   float64
 16  w2chn    5127 non-null   float32
 17  w2mat    5127 non-null   float32
 18  w2eng    5127 non-null   float32
dtypes: float32(4), float64(11), int16(3), int8(1)
memory usage: 595.8 KB
None


In [15]:
data_.rename(columns={'a13':'1314身高','a14':'1314体重','a15':'是否近视','w2cogscore':'认知测试原始总分','w2cog3pl':'认知测试标准化得分',
                           'w2chn':'期中语文','w2mat':'期中数学','w2eng':'期中英语'},inplace=True)
data_des = data_.describe()

In [16]:
data_des.drop(['ids','clsids','schids','ctyids','frame','subsample','fall','sweight','w2frame','w2subsample','w2fall','w2sweight'],axis=1,inplace=True)
data_des

,1314身高,1314体重,是否近视,认知测试原始总分,认知测试标准化得分,期中语文,期中数学,期中英语
count,5787.000000,5787.000000,5787.000000,5787.000000,5787.000000,5787.000000,5787.000000,5787.000000
mean,158.592535,93.912390,0.525488,22.639710,0.242947,74.497406,65.303009,64.326912
std,8.472717,22.390991,0.499393,6.410028,0.825728,15.454585,25.870459,24.322811
min,130.000000,50.000000,0.000000,0.000000,-3.136649,0.000000,0.000000,0.000000
25%,153.000000,80.000000,0.000000,18.000000,-0.304220,67.000000,46.000000,45.000000
50%,159.000000,90.000000,1.000000,23.000000,0.361397,77.000000,72.000000,69.500000
75%,165.000000,104.000000,1.000000,28.000000,0.839252,85.000000,87.000000,86.000000
max,190.000000,210.000000,1.000000,35.000000,2.062761,100.000000,100.000000,100.000000


##### For x,y and control variables

In [9]:
data_xy.rename(columns={'stsex':'性别','a13':'1314身高','a14':'1314体重','a15':'是否近视','cog3pl':'基线认知测试标准分','sthktype':'户口类型',
                           'stonly':'是否独生子女','steco_5c':'经济条件','b31':'父母教育期望','stmedu':'母亲教育程度','stfedu':'父亲教育程度',
                        'w2a0801':'父母婚姻状况','w2b18':'自己教育期望','w2chn':'期中语文','w2mat':'期中数学','w2eng':'期中英语'},inplace=True)
data_xy['平均分']=(data_xy['期中语文']+data_xy['期中数学']+data_xy['期中英语'])/3
data_xy.drop(['ids','clsids','schids','ctyids'],axis=1,inplace=True)

In [10]:
data_xy.describe()

,性别,1314身高,1314体重,是否近视,基线认知测试标准分,户口类型,是否独生子女,经济条件,父母教育期望,母亲教育程度,父亲教育程度,父母婚姻状况,自己教育期望,期中语文,期中数学,期中英语,BMI,平均分
count,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000,5702.000000
mean,0.551035,1.586170,46.981147,0.524202,0.006833,0.526307,1.540512,2.821642,6.809014,3.921256,4.279551,0.915819,6.773939,74.501404,65.310509,64.280312,18.558501,68.030731
std,0.497432,0.084166,11.174500,0.499458,0.857742,0.499351,0.498400,0.606000,1.667240,1.974742,1.991580,0.277683,1.757915,15.516328,25.890614,24.306482,3.576001,18.725079
min,0.000000,1.300000,25.000000,0.000000,-2.028966,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,8.549639,0.000000
25%,0.000000,1.530000,40.000000,0.000000,-0.561718,0.000000,1.000000,3.000000,6.000000,3.000000,3.000000,1.000000,6.000000,67.000000,46.000000,45.000000,16.406250,55.333332
50%,1.000000,1.590000,45.000000,1.000000,-0.019050,1.000000,2.000000,3.000000,7.000000,3.000000,3.000000,1.000000,7.000000,77.000000,72.000000,69.000000,17.968750,72.333336
75%,1.000000,1.650000,52.000000,1.000000,0.606752,1.000000,2.000000,3.000000,8.000000,6.000000,6.000000,1.000000,8.000000,85.000000,87.000000,85.500000,20.060954,83.500000
max,1.000000,1.900000,105.000000,1.000000,2.333043,1.000000,2.000000,5.000000,10.000000,9.000000,9.000000,1.000000,10.000000,100.000000,100.000000,100.000000,55.144033,99.000000


In conclusion:<br>
55% of the 5702 student samples are boys;<br>
52% of the 5702 student samples are short-sighted;<br>
52% of the 5702 student samples has agricultural account;<br>
92% of the 5702 student samples has married parents;<br>
The average economy circumstance is middle;<br>
The average parental and self expectation for children's education is 7 —— university students;<br>
Note:<br>
heights are in meters, weights are in kgs, 0 represents yes, 1 represents no.<br>
Other grading policy please refer to **the data manual**.


### Regression Analysis

In [16]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [19]:
data_.drop(['ids','clsids','schids','ctyids','frame','subsample','fall','sweight','w2frame','w2subsample','w2fall','w2sweight'],axis=1,inplace=True)

In [21]:
X = data_.copy()
X

,1314身高,1314体重,是否近视,认知测试原始总分,认知测试标准化得分,期中语文,期中数学,期中英语
0,154.0,76.0,1.0,26,0.544194,82.0,91.0,82.0
1,169.0,90.0,0.0,26,0.898057,95.0,63.0,83.0
2,176.0,120.0,1.0,29,0.842200,89.0,68.0,74.0
3,160.0,90.0,1.0,17,-0.288543,85.0,80.0,63.0
4,167.0,150.0,1.0,30,1.143127,91.0,73.0,84.0
...,...,...,...,...,...,...,...,...
10273,151.0,84.0,0.0,20,-0.154441,79.0,26.0,48.5
10274,151.0,92.0,1.0,19,-0.073807,89.0,72.0,61.5
10275,158.0,98.0,1.0,11,-0.803730,78.0,29.0,44.0
10277,151.0,80.0,0.0,22,-0.006150,87.0,21.0,67.5


In [22]:
# resize height to m, resize weight to kg,calculate BMI
X['1314身高'] = X['1314身高'].apply(lambda x:x/100)
X['1314体重'] = X['1314体重'].apply(lambda x:x/2)
X['BMI'] = X['1314体重']/(X['1314身高']*X['1314身高'])
X.head()

,1314身高,1314体重,是否近视,认知测试原始总分,认知测试标准化得分,期中语文,期中数学,期中英语,BMI
0,1.54,38.0,1.0,26,0.544194,82.0,91.0,82.0,16.022938
1,1.69,45.0,0.0,26,0.898057,95.0,63.0,83.0,15.755751
2,1.76,60.0,1.0,29,0.842200,89.0,68.0,74.0,19.369835
3,1.60,45.0,1.0,17,-0.288543,85.0,80.0,63.0,17.578125
4,1.67,75.0,1.0,30,1.143127,91.0,73.0,84.0,26.892323


In [23]:
Y = X[['认知测试原始总分','认知测试标准化得分','期中语文','期中数学','期中英语']]
X = X[['BMI','是否近视']]
Y.head()

,认知测试原始总分,认知测试标准化得分,期中语文,期中数学,期中英语
0,26,0.544194,82.0,91.0,82.0
1,26,0.898057,95.0,63.0,83.0
2,29,0.842200,89.0,68.0,74.0
3,17,-0.288543,85.0,80.0,63.0
4,30,1.143127,91.0,73.0,84.0


In [27]:
X = X[['BMI','是否近视']]
Y.head()
Y['平均分']=(Y['期中语文']+Y['期中数学']+Y['期中英语'])/3
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)
ychn_train = y_train[['期中语文']]
ymat_train = y_train[['期中数学']]
yeng_train = y_train[['期中英语']]
yavg_train = y_train[['平均分']]
ychn_test = y_test[['期中语文']]
ymat_test = y_test[['期中数学']]
yeng_test = y_test[['期中英语']]
yavg_test = y_test[['平均分']]
yavg_train.head()

<ipython-input-27-1beb0de11ac6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['平均分']=(Y['期中语文']+Y['期中数学']+Y['期中英语'])/3


,平均分
4262,42.666668
830,81.666664
6814,84.833336
655,89.000000
6017,68.000000


### Linear Regression on Chiness Score

In [28]:
regr_chn = linear_model.LinearRegression()
regr_chn.fit(X_train,ychn_train)
print(regr_chn.intercept_)
print(regr_chn.coef_)
print("train score:",regr_chn.score(X_train,ychn_train))
print("test score:",regr_chn.score(X_test,ychn_test))

[73.49770474]
[[-0.05214931  4.11147682]]
train score: 0.01798462076861751
test score: 0.011157286787125797


### Linear Regression on English Score

In [29]:
regr_eng = linear_model.LinearRegression()
regr_eng.fit(X_train,yeng_train)
print(regr_eng.intercept_)
print(regr_eng.coef_)
print("train score:",regr_eng.score(X_train,yeng_train))
print("test score:",regr_chn.score(X_test,yeng_test))

[57.26917809]
[[0.12786015 8.94830936]]
train score: 0.03458156418101799
test score: -0.16030120770768663


### Linear Regression on Math Score

In [30]:
regr_mat = linear_model.LinearRegression()
regr_mat.fit(X_train,ymat_train)
print(regr_mat.intercept_)
print(regr_mat.coef_)
print("train score:",regr_mat.score(X_train,ymat_train))
print("test score:",regr_mat.score(X_test,ymat_test))

[56.49752928]
[[0.26491977 7.57976036]]
train score: 0.023107524119044487
test score: 0.014438525330832341


### Linear Regression on Average Score

In [31]:
regr_avg = linear_model.LinearRegression()
regr_avg.fit(X_train,yavg_train)
print(regr_avg.intercept_)
print(regr_avg.coef_)
print("train score:",regr_avg.score(X_train,yavg_train))
print("test score:",regr_avg.score(X_test,yavg_test))

[62.42147059]
[[0.11354354 6.87984889]]
train score: 0.03497434748060246
test score: 0.021953132137419562


### Linear Regression on X and other control variable

In [32]:
XC = data_xy[['性别','BMI','是否近视','基线认知测试标准分','户口类型','是否独生子女','经济条件','父母教育期望','母亲教育程度','父亲教育程度',
              '父母婚姻状况','自己教育期望']]
YC = data_xy[['平均分']]
XC_train, XC_test, yc_train, yc_test = train_test_split(XC, YC, random_state=1)
regr_c = linear_model.LinearRegression()
regr_c.fit(XC_train,yc_train)
print(regr_c.intercept_)
print(regr_c.coef_)
print("train score:",regr_c.score(XC_train,yc_train))
print("test score:",regr_c.score(XC_test,yc_test))

[47.86116963]
[[-6.76763603 -0.1287171   2.5475935   7.47232958 -0.22934683 -2.42923538
   0.33163507  1.35437014  0.22791818  0.46393134  2.86651939  1.9780406 ]]
train score: 0.3489845892235264
test score: 0.39700370996380474


The most affective variable is the '标准化认知测试成绩‘, which makes sense since cognitive testing score
is direcly related to mid-term test scores.<br>
The second affective variable is gender, indicating girls tend to have better scores.<br>
The coefficient of BMI suggests that the higher BMI, the lower the test score will be.<br>
The coefficient of eye-sight situation suggests if one has poor eye-sight, he or she might
have higher testing scores.

#### Group Comparison Analysis
| 等级  | 标准        |
|-----|-----------|
| 低体重 | <=17.1    |
| 正常  | 17.2~23.9 |
| 超重  | 24.0~27.9 |
| 肥胖  | >=28.0    |

In [11]:
data_xy_sort = data_xy.sort_values(by=['BMI'])
data_low = data_xy_sort.loc[data_xy_sort['BMI']<17.2]
data_norm = data_xy_sort.loc[(data_xy_sort['BMI']<24) & (data_xy_sort['BMI']>=17.2)]
data_over = data_xy_sort.loc[(data_xy_sort['BMI']<28) & (data_xy_sort['BMI']>=24)]
data_fat = data_xy_sort.loc[data_xy_sort['BMI']>=28]
print('Number of student with low weight: ',data_low.shape[0])
print('Number of student with normal weight: ',data_norm.shape[0])
print('Number of student over weight: ',data_over.shape[0])
print('Number of student too fat: ',data_fat.shape[0])

Number of student with low weight:  2132
Number of student with normal weight:  3170
Number of student over weight:  277
Number of student too fat:  123


### Descriptive Analysis

In [12]:
data_low.describe()

,性别,1314身高,1314体重,是否近视,基线认知测试标准分,户口类型,是否独生子女,经济条件,父母教育期望,母亲教育程度,父亲教育程度,父母婚姻状况,自己教育期望,期中语文,期中数学,期中英语,BMI,平均分
count,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000
mean,0.523921,1.568166,38.228424,0.489212,-0.069674,0.566604,1.598030,2.772514,6.732645,3.712946,4.098030,0.914634,6.702158,74.105766,63.526501,62.526245,15.513833,66.719513
std,0.499545,0.083213,5.260763,0.500001,0.848833,0.495660,0.490411,0.625093,1.682104,1.854177,1.892527,0.279491,1.771545,16.003927,26.083803,24.338112,1.439028,18.800076
min,0.000000,1.300000,25.000000,0.000000,-2.028966,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,8.549639,0.000000
25%,0.000000,1.500000,35.000000,0.000000,-0.618943,0.000000,1.000000,3.000000,6.000000,3.000000,3.000000,1.000000,6.000000,67.000000,43.000000,42.000000,14.860540,53.333332
50%,1.000000,1.570000,39.000000,0.000000,-0.081955,1.000000,2.000000,3.000000,7.000000,3.000000,3.000000,1.000000,7.000000,76.000000,69.000000,67.000000,15.816857,70.666664
75%,1.000000,1.620000,42.000000,1.000000,0.504647,1.000000,2.000000,3.000000,8.000000,5.000000,6.000000,1.000000,8.000000,85.000000,86.000000,84.000000,16.608997,82.333336
max,1.000000,1.900000,59.000000,1.000000,2.333043,1.000000,2.000000,5.000000,10.000000,9.000000,9.000000,1.000000,10.000000,100.000000,100.000000,100.000000,17.187500,99.000000


In [13]:
data_norm.describe()

,性别,1314身高,1314体重,是否近视,基线认知测试标准分,户口类型,是否独生子女,经济条件,父母教育期望,母亲教育程度,父亲教育程度,父母婚姻状况,自己教育期望,期中语文,期中数学,期中英语,BMI,平均分
count,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000
mean,0.547319,1.592833,49.604890,0.543218,0.046838,0.514511,1.517035,2.841009,6.846372,3.998423,4.345426,0.917035,6.809779,74.928230,66.428078,65.676971,19.486890,69.011093
std,0.497834,0.081067,6.980145,0.498207,0.854591,0.499868,0.499789,0.590198,1.651585,2.020797,2.029038,0.275873,1.740347,15.101841,25.725216,24.121908,1.713896,18.550646
min,0.000000,1.300000,30.000000,0.000000,-2.028966,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,17.206598,6.333333
25%,0.000000,1.540000,45.000000,0.000000,-0.538550,0.000000,1.000000,3.000000,6.000000,3.000000,3.000000,1.000000,6.000000,68.000000,48.000000,47.000000,18.048036,57.000000
50%,1.000000,1.600000,49.000000,1.000000,0.017257,1.000000,2.000000,3.000000,7.000000,3.000000,3.000000,1.000000,7.000000,77.500000,74.000000,71.500000,19.111111,73.333336
75%,1.000000,1.650000,54.000000,1.000000,0.662354,1.000000,2.000000,3.000000,8.000000,6.000000,6.000000,1.000000,8.000000,85.000000,88.000000,86.000000,20.570904,84.000000
max,1.000000,1.900000,85.000000,1.000000,2.333043,1.000000,2.000000,5.000000,10.000000,9.000000,9.000000,1.000000,10.000000,100.000000,100.000000,100.000000,23.999459,98.666664


In [14]:
data_over.describe()

,性别,1314身高,1314体重,是否近视,基线认知测试标准分,户口类型,是否独生子女,经济条件,父母教育期望,母亲教育程度,父亲教育程度,父母婚姻状况,自己教育期望,期中语文,期中数学,期中英语,BMI,平均分
count,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000
mean,0.714801,1.631516,68.568592,0.563177,0.109085,0.379061,1.393502,2.935018,6.978339,4.624549,4.974729,0.920578,6.902527,73.792419,67.299637,64.557404,25.678767,68.549820
std,0.452326,0.086035,8.017123,0.496890,0.898034,0.486032,0.489411,0.598399,1.650505,2.115294,2.130543,0.270886,1.719842,15.251077,24.802124,24.509130,1.131921,18.722364
min,0.000000,1.370000,46.500000,0.000000,-2.008923,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,10.000000,3.000000,12.000000,24.000000,11.666667
25%,0.000000,1.580000,63.000000,0.000000,-0.449563,0.000000,1.000000,3.000000,6.000000,3.000000,3.000000,1.000000,6.000000,67.000000,53.000000,44.000000,24.654832,57.666668
50%,1.000000,1.630000,69.500000,1.000000,0.178986,0.000000,1.000000,3.000000,7.000000,4.000000,6.000000,1.000000,7.000000,76.000000,74.000000,71.000000,25.477925,73.000000
75%,1.000000,1.690000,74.000000,1.000000,0.690563,1.000000,2.000000,3.000000,8.000000,6.000000,7.000000,1.000000,8.000000,84.000000,88.000000,86.000000,26.672763,83.000000
max,1.000000,1.860000,93.000000,1.000000,2.081214,1.000000,2.000000,5.000000,10.000000,9.000000,9.000000,1.000000,10.000000,100.000000,100.000000,100.000000,27.929687,98.666664


In [15]:
data_fat.describe()

,性别,1314身高,1314体重,是否近视,基线认知测试标准分,户口类型,是否独生子女,经济条件,父母教育期望,母亲教育程度,父亲教育程度,父母婚姻状况,自己教育期望,期中语文,期中数学,期中英语,BMI,平均分
count,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000
mean,0.747967,1.624390,82.459350,0.552846,0.071652,0.463415,1.479675,2.918699,6.788618,3.959350,4.162602,0.894309,6.804878,71.955284,62.951218,58.065041,31.370896,64.323845
std,0.435956,0.098298,9.793062,0.499233,0.909726,0.500699,0.501630,0.621915,1.802785,1.985184,1.943353,0.308699,2.014938,17.640421,27.483484,25.390715,4.146134,20.241892
min,0.000000,1.300000,52.000000,0.000000,-1.981191,0.000000,1.000000,1.000000,2.000000,1.000000,2.000000,0.000000,1.000000,5.000000,6.000000,12.000000,28.010224,14.666667
25%,0.500000,1.575000,75.000000,0.000000,-0.566067,0.000000,1.000000,3.000000,6.000000,3.000000,3.000000,1.000000,6.000000,63.000000,38.500000,35.000000,28.959000,48.000000
50%,1.000000,1.620000,81.000000,1.000000,0.103304,0.000000,1.000000,3.000000,7.000000,3.000000,3.000000,1.000000,7.000000,76.000000,69.000000,60.000000,29.568411,68.333336
75%,1.000000,1.695000,90.000000,1.000000,0.722478,1.000000,2.000000,3.000000,8.000000,6.000000,6.000000,1.000000,8.000000,84.250000,87.000000,81.500000,32.640287,81.666668
max,1.000000,1.810000,105.000000,1.000000,2.073272,1.000000,2.000000,5.000000,10.000000,9.000000,9.000000,1.000000,10.000000,96.500000,100.000000,100.000000,55.144033,93.166664


### Regression Analysis
#### Low Weight Group

In [17]:
XC_low = data_low[['性别','BMI','是否近视','基线认知测试标准分','户口类型','是否独生子女','经济条件','父母教育期望','母亲教育程度','父亲教育程度',
              '父母婚姻状况','自己教育期望']]
YC_low = data_low[['平均分']]
XClow_train, XClow_test, yclow_train, yclow_test = train_test_split(XC_low, YC_low, random_state=1)
regrlow_c = linear_model.LinearRegression()
regrlow_c.fit(XClow_train,yclow_train)
print(regrlow_c.intercept_)
print(regrlow_c.coef_)
print("train score:",regrlow_c.score(XClow_train,yclow_train))
print("test score:",regrlow_c.score(XClow_test,yclow_test))

[40.44804576]
[[-8.13539803  0.25724775  3.0375898   7.66387068  0.32198926 -1.29429944
  -0.55797816  1.33433505  0.36662769  0.40938799  3.97760163  2.02867747]]
train score: 0.35584928841122077
test score: 0.34923349438436846


#### Normal Weight Group

In [18]:
XC_norm = data_norm[['性别','BMI','是否近视','基线认知测试标准分','户口类型','是否独生子女','经济条件','父母教育期望','母亲教育程度','父亲教育程度',
              '父母婚姻状况','自己教育期望']]
YC_norm = data_norm[['平均分']]
XCnorm_train, XCnorm_test, ycnorm_train, ycnorm_test = train_test_split(XC_norm, YC_norm, random_state=1)
regrnorm_c = linear_model.LinearRegression()
regrnorm_c.fit(XCnorm_train,ycnorm_train)
print(regrnorm_c.intercept_)
print(regrnorm_c.coef_)
print("train score:",regrnorm_c.score(XCnorm_train,ycnorm_train))
print("test score:",regrnorm_c.score(XCnorm_test,ycnorm_test))

[45.57138631]
[[-6.81308466 -0.07812112  3.04308707  7.28457619 -0.81560114 -2.63682364
   1.4470703   1.31995927  0.04279306  0.43231794  1.77418106  2.10404118]]
train score: 0.3712995505808835
test score: 0.3639738059616683


#### Over Weight Group

In [19]:
XC_over = data_over[['性别','BMI','是否近视','基线认知测试标准分','户口类型','是否独生子女','经济条件','父母教育期望','母亲教育程度','父亲教育程度',
              '父母婚姻状况','自己教育期望']]
YC_over = data_over[['平均分']]
XCover_train, XCover_test, ycover_train, ycover_test = train_test_split(XC_over, YC_over, random_state=1)
regrover_c = linear_model.LinearRegression()
regrover_c.fit(XCover_train,ycover_train)
print(regrover_c.intercept_)
print(regrover_c.coef_)
print("train score:",regrover_c.score(XCover_train,ycover_train))
print("test score:",regrover_c.score(XCover_test,ycover_test))

[95.00545931]
[[-3.09168819 -2.28320496  1.06673795  5.56798319 -1.93953867 -0.70604297
  -0.36756387  2.38474654  0.17284307  2.07567871 -0.94585794  1.25649527]]
train score: 0.39080717398123843
test score: 0.3084211842521327


#### Fat Group

In [20]:
XC_fat = data_fat[['性别','BMI','是否近视','基线认知测试标准分','户口类型','是否独生子女','经济条件','父母教育期望','母亲教育程度','父亲教育程度',
              '父母婚姻状况','自己教育期望']]
YC_fat = data_fat[['平均分']]
XCfat_train, XCfat_test, ycfat_train, ycfat_test = train_test_split(XC_fat, YC_fat, random_state=1)
regrfat_c = linear_model.LinearRegression()
regrfat_c.fit(XCfat_train,ycfat_train)
print(regrfat_c.intercept_)
print(regrfat_c.coef_)
print("train score:",regrfat_c.score(XCfat_train,ycfat_train))
print("test score:",regrfat_c.score(XCfat_test,ycfat_test))

[64.5469173]
[[-0.35883527 -0.85837751  3.62498562  5.17725792  6.09706063 -6.67488698
  -4.99081341  1.09367506  2.26346749  0.3704375   0.86063963  4.00369936]]
train score: 0.5742433032568801
test score: -0.052076799907464055


#### Comparison result
1. For low weight group, the heavier the student, the higher the test score;<br>
2. For low weight and normal weight group, if they are short-sighted, they tend to achieve
better scores;
3. For over weight and fat group, the heavier they are, the lower their scores are.